<a href="https://colab.research.google.com/github/diptoromeo/BERT-and-ELMo-with-CNN-BiLSTM-and-CNN_BiLSTM/blob/main/BERT_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import time
import datetime
import gc
from random import seed
import nltk
import os
import pandas as pd
import regex
import numpy as np
from keras.utils import pad_sequences
from numpy.f2py.crackfortran import quiet
from opt_einsum.backends import torch
from pandas.core.common import flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from string import punctuation
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
# ================================================================================================
from transformers import BertModel,BertForSequenceClassification, AdamW, BertConfig,BertTokenizer,get_linear_schedule_with_warmup
import torch
import torch.nn as nn
!pip install pytorch_lightning
import pytorch_lightning as pl
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
from sklearn.metrics import roc_auc_score,f1_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.7/777.7 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 68.0 MB/s eta 0:00:00


In [2]:
# cup cuda checking
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
# ===============================Nltk abstract_words tokenize======================================
with open('/content/FGFSJournal.txt', 'rt', encoding='UTF8') as file:
    FGFS_abstract = []
    for line in file:
        if '<abstract>' in line:
            abstract = line.split('</abstract>')[0].split('<abstract>')[-1]
            abstract = ''.join(i for i in abstract if not i.isdigit())
            abstract = regex.sub('[^\w\d\s]+', '', abstract)
            ##abstract = nltk.sent_tokenize(abstract)
            abstract = nltk.word_tokenize(abstract)
            stop_words = set(stopwords.words('english'))
            filtered_sentence_abstract = [w.lower() for w in abstract if
                                          w.lower() not in punctuation and w.lower() not in stop_words]
            tagged_list = nltk.pos_tag(filtered_sentence_abstract)
            nouns_list = [t[0] for t in tagged_list if t[-1] == 'NN']
            lm = WordNetLemmatizer()
            singluar_form = [lm.lemmatize(w, pos='v') for w in nouns_list]
            FGFS_abstract.append(singluar_form)

print("FGCS data:", len(FGFS_abstract))


FGCS data: 5659


In [4]:
#======================================================train_labels==========================================================================
ten_words = ['paper', 'system', 'performance', 'network', 'model', 'service', 'time', 'information', 'approach', 'cloud']
twenty_words = ['paper', 'system', 'performance', 'network', 'model', 'service', 'time', 'information', 'approach', 'cloud',
                'problem', 'process', 'security', 'analysis', 'application', 'method', 'research', 'framework', 'number', 'resource']
thirty_words = ['paper', 'system', 'performance', 'network', 'model', 'service', 'time', 'information', 'approach', 'cloud',
                'problem', 'process', 'security', 'analysis', 'application', 'method', 'research', 'framework', 'number', 'resource',
               'environment', 'algorithm', 'energy', 'management', 'architecture', 'access', 'scheme', 'communication', 'execution', 'order']


counts = 3

##==============================10-words label==================================
ten_words_labels = []

for i in range(0, 5659):
    count = 0
    for j in range(0, len(ten_words)):
        if ten_words[j] in FGFS_abstract[i]:
            count += 1
    if count >=counts:
        ten_words_labels.append(1)
    else:
        ten_words_labels.append(0)

print("ten_words_labels:", len(ten_words_labels))

##==============================20-words label==================================
twenty_words_labels = []

for i in range(0, 5659):
    count = 0
    for j in range(0, len(twenty_words)):
        if twenty_words[j] in FGFS_abstract[i]:
            count += 1
    if count >=counts:
        twenty_words_labels.append(1)
    else:
        twenty_words_labels.append(0)

print("twenty_words_labels:", len(twenty_words_labels))

##==============================30-words label==================================
thrity_words_labels = []

for i in range(0, 5659):
    count = 0
    for j in range(0, len(thirty_words)):
        if thirty_words[j] in FGFS_abstract[i]:
            count += 1
    if count >=counts:
        thrity_words_labels.append(1)
    else:
        thrity_words_labels.append(0)

print("thrity_words_labels:", len(thrity_words_labels))


# ##==============================10-words Multi-label==================================
# ten_words_labels = []

# for doc in FGFS_abstract:
#     label = []
#     for term in ten_words:
#         if term in doc:
#             label.append(1)
#         else:
#             label.append(0)
#     ten_words_labels.append(label)

# print("ten_words_labels:", len(ten_words_labels))

# ##==============================20-words Multi-label==================================
# twenty_words_labels = []

# for doc in FGFS_abstract:
#     label = []
#     for term in twenty_words:
#         if term in doc:
#             label.append(1)
#         else:
#             label.append(0)
#     twenty_words_labels.append(label)

# print("twenty_words_labels:", len(twenty_words_labels))

# ##==============================30-words Multi-label==================================
# thrity_words_labels = []
# for doc in FGFS_abstract:
#     label = []
#     for term in ten_words:
#         if term in doc:
#             label.append(1)
#         else:
#             label.append(0)
#     thrity_words_labels.append(label)

# print("thrity_words_labelss:", len(thrity_words_labels))

ten_words_labels: 5659
twenty_words_labels: 5659
thrity_words_labels: 5659


In [5]:
##=================================Bert_fine-tuning for sequence model==========================
# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# cup cuda checking
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# ===============================================================================================
# checking the sentences line that is 156
max_len = 0

# For every sentence...
for sent in FGFS_abstract:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

##===============================================================================================

# Doing attention masking
input_ids = []
attention_masks = []

# For every abstracts...
for article in FGFS_abstract:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
        article,  # Sentence to encode.
        add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
        max_length=max_len,  # Pad & truncate all sentences.
        pad_to_max_length=True,
        return_attention_mask=True,  # Construct attn. masks.
        return_tensors='pt',  # Return pytorch tensors.
    )

    # Add the encoded sentence to the list.
    input_ids.append(encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
ten_labels = torch.tensor(ten_words_labels)
twenty_labels = torch.tensor(twenty_words_labels)
thrity_labels = torch.tensor(thrity_words_labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', article[0])
print('Token IDs:', input_ids[0])


##=========================================================================
ten_words_labels = torch.tensor(ten_words_labels)
twenty_words_labels = torch.tensor(twenty_words_labels)
thrity_words_labels = torch.tensor(thrity_words_labels)

# Combine the training inputs into a TensorDataset.
dataset1 = TensorDataset(input_ids, attention_masks, ten_words_labels)
dataset2 = TensorDataset(input_ids, attention_masks, twenty_words_labels)
dataset3 = TensorDataset(input_ids, attention_masks, thrity_words_labels)

# Create a 70-30 train-validation split.
# Calculate the number of 10_words_samples to include in each set.
train_size = int(0.7 * len(dataset1))
val_size = int(0.3 * len(dataset1))
val_size = len(dataset1) - train_size

# Calculate the number of 20_words_samples to include in each set.
train_size = int(0.8 * len(dataset2))
val_size = int(0.2 * len(dataset2))
val_size = len(dataset2) - train_size

# Calculate the number of 30_words_samples to include in each set.
train_size = int(0.8 * len(dataset3))
val_size = int(0.2 * len(dataset3))
val_size = len(dataset3) - train_size



# Divide the dataset by randomly selecting samples.
train_dataset1, val_dataset1 = random_split(dataset1, [train_size, val_size])
train_dataset2, val_dataset2 = random_split(dataset2, [train_size, val_size])
train_dataset3, val_dataset3 = random_split(dataset3, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

##===============================================================================================
# The DataLoader needs to know our batch size for training, so we specify it
# here. For fine-tuning BERT on a specific task, the authors recommend a batch
# size of 16 or 32.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training 10_words samples in random order.
train_dataloader1 = DataLoader(
            train_dataset1,  # The training samples.
            sampler = RandomSampler(train_dataset1), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader1 = DataLoader(
            val_dataset1, # The validation samples.
            sampler = SequentialSampler(val_dataset1), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

##================================================================================================
# We'll take training 20_words samples in random order.
train_dataloader2 = DataLoader(
            train_dataset2,  # The training samples.
            sampler = RandomSampler(train_dataset2), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader2 = DataLoader(
            val_dataset2, # The validation samples.
            sampler = SequentialSampler(val_dataset2), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

##================================================================================================
# We'll take training 30_words samples in random order.
train_dataloader3 = DataLoader(
            train_dataset3,  # The training samples.
            sampler = RandomSampler(train_dataset3), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader3 = DataLoader(
            val_dataset3, # The validation samples.
            sampler = SequentialSampler(val_dataset3), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

cuda:0


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Max sentence length:  156


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  growth
Token IDs: tensor([  101,  4294, 28937,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,

In [6]:
# ===========================================================================
tuned_model = BertForSequenceClassification.from_pretrained('bert-base-uncased')


# Load BertForSequenceClassification, the pretrained BERT model with a single
class BertCNNClassifier(nn.Module):
    def __init__(self, tuned_model, embed_num = 512, embed_dim = 768, dropout=0.1, kernel_num=3, kernel_sizes=[1,2], num_labels=2):
        super().__init__()
        self.num_labels = num_labels
        self.embed_num = embed_num
        self.embed_dim = embed_dim
        self.dropout = dropout
        self.kernel_num = kernel_num
        self.kernel_sizes = kernel_sizes
        self.softmax = nn.functional.softmax

        self.bert = tuned_model.bert
        self.convs = nn.ModuleList([nn.Conv2d(1, self.kernel_num, (k, self.embed_dim)) for k in self.kernel_sizes])
        self.dropout = nn.Dropout(self.dropout)
        self.classifier = nn.Linear(len(self.kernel_sizes)*self.kernel_num, self.num_labels)

    def forward(self, input_ids, attention_mask, token_type_ids = None):
        output = self.bert(input_ids = input_ids, attention_mask = attention_mask, token_type_ids = token_type_ids) #16,512,768
        output = output[0].unsqueeze(1) #16,1,512,768
        output = [nn.functional.relu(conv(output)).squeeze(3) for conv in self.convs] #16,3,508,1 => #16,3,508
        output = [nn.functional.max_pool1d(i, i.size(2)).squeeze(2) for i in output] #=> 16,3
        output = torch.cat(output, 1)
        output = self.dropout(output)
        logits = self.classifier(output)
        return self.softmax(logits, 1)

# ======================================================================================

# Initializing model
model1 = BertCNNClassifier(tuned_model=tuned_model)
model1.to(device)

# ================================================================================
# set parameters
epochs = 4

optimizer = torch.optim.AdamW(model1.parameters(),
                  lr=5e-5,  # args.learning_rate - default is 5e-5.
                  eps=1e-8  # args.adam_epsilon  - default is 1e-8.
                )
criterion = nn.CrossEntropyLoss()
# ===========================Fine-tuning model===========================================
# Number of training epochs. The BERT authors recommend between 2 and 4.
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 4

# 10_words number of training steps is [number of batches] x [number of epochs].
# (Note that this is not the same as the number of training samples).
total_steps1 = len(train_dataloader1) * epochs

# Create the learning rate scheduler.
scheduler1 = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,  # Default value in run_glue.py
                                            num_training_steps=total_steps1)

##================================================================================================
# 20_words number of training steps is [number of batches] x [number of epochs].
total_steps2 = len(train_dataloader2) * epochs

# Create the learning rate scheduler.
scheduler2 = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,  # Default value in run_glue.py
                                            num_training_steps=total_steps2)

##================================================================================================
# 30_words number of training steps is [number of batches] x [number of epochs].
total_steps3 = len(train_dataloader3) * epochs

# Create the learning rate scheduler.
scheduler3 = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,  # Default value in run_glue.py
                                            num_training_steps=total_steps3)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERT + CNN with Ten_Words Labels

In [22]:

##=====================================================================================
seed_val = 42
random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []
total_t0 = time.time()
best_accuracy = 0
for epoch_i in range(0, epochs):
    #Training
    print("")
    print('Epoch {:} / {:}'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_train_loss = 0
    total_train_accuracy = 0
    model1.train()
    for step, batch in enumerate(train_dataloader1):

        input_ids = batch[0].to(device)
        input_mask = batch[1].to(device)
        labels = batch[2].to(device)

        model1.zero_grad()
        out = model1(input_ids = input_ids, attention_mask = input_mask, token_type_ids = None)
        loss = criterion(out, labels)
        total_train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model1.parameters(), 1.0)
        optimizer.step()

        pred = torch.argmax(out, dim = 1)
        total_train_accuracy +=  torch.sum(pred == labels).item()

    avg_train_accuracy = total_train_accuracy / len(train_dataloader1)
    avg_train_loss = total_train_loss / len(train_dataloader1)
    print("  BERT+CNN_10_Words Accuracy: {}".format(avg_train_accuracy))
    print("  BERT+CNN_10_Words Training loss: {}".format(avg_train_loss))

    # Validation
    print("")
    print("Validation...")
    model1.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    y_true = []
    y_pred = []

    for batch in validation_dataloader1:
        input_ids = batch[0].to(device)
        input_mask = batch[1].to(device)
        labels = batch[2].to(device)

        with torch.no_grad():
            out = model1(input_ids = input_ids, attention_mask = input_mask, token_type_ids = None)
        loss = criterion(out, labels)
        total_eval_loss += loss.item()
        pred = torch.argmax(out, dim = 1)
        total_eval_accuracy += torch.sum(pred == labels).item()
        y_true.append(labels.flatten())
        y_pred.append(pred.flatten())

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader1)
    print("  BERT+CNN_10_Words Accuracy: {}".format(avg_val_accuracy))
    avg_val_loss = total_eval_loss / len(validation_dataloader1)
    print("  BERT+CNN_10_Words Validation loss: {}".format(avg_val_loss))
    training_time = format_time(time.time() - t0)
    print("  BERT+CNN_10_Words This epoch took: {:}".format(training_time))
    print()
    y_true = torch.cat(y_true).tolist()
    y_pred = torch.cat(y_pred).tolist()
    print('  BERT+CNN_10_Words roc_auc score: ', roc_auc_score(y_true,y_pred))
    print('  BERT+CNN_10_Words F1 score:',f1_score(y_true, y_pred))


    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Train Accur.': avg_train_accuracy,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
        }
    )

    if avg_val_accuracy > best_accuracy:
        best_accuracy = avg_val_accuracy
        best_model = model1

print("===")
print("Summary")
print("BERT+CNN_10_Words Total time {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
print('BERT+CNN_10_Words best acc:',best_accuracy)


Epoch 1 / 4
Training...
  BERT+CNN_10_Words Accuracy: 14.657243816254416
  BERT+CNN_10_Words Training loss: 0.4015863109182553

Validation...
  BERT+CNN_10_Words Accuracy: 14.71830985915493
  BERT+CNN_10_Words Validation loss: 0.3905222457899174
  BERT+CNN_10_Words This epoch took: 0:02:20

  BERT+CNN_10_Words roc_auc score:  0.9040232146370254
  BERT+CNN_10_Words F1 score: 0.9402061855670104

Epoch 2 / 4
Training...
  BERT+CNN_10_Words Accuracy: 14.996466431095406
  BERT+CNN_10_Words Training loss: 0.3762343172049775

Validation...
  BERT+CNN_10_Words Accuracy: 14.985915492957746
  BERT+CNN_10_Words Validation loss: 0.3739074317502304
  BERT+CNN_10_Words This epoch took: 0:02:19

  BERT+CNN_10_Words roc_auc score:  0.9242966751918159
  BERT+CNN_10_Words F1 score: 0.9528432732316228

Epoch 3 / 4
Training...
  BERT+CNN_10_Words Accuracy: 15.303886925795053
  BERT+CNN_10_Words Training loss: 0.3565075214254561

Validation...
  BERT+CNN_10_Words Accuracy: 15.32394366197183
  BERT+CNN_10_

BERT+CNN with Twenty_Words Labels

In [23]:
##=====================================================================================
seed_val = 42
random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []
total_t0 = time.time()
best_accuracy = 0
for epoch_i in range(0, epochs):
    #Training
    print("")
    print('Epoch {:} / {:}'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_train_loss = 0
    total_train_accuracy = 0
    model1.train()
    for step, batch in enumerate(train_dataloader2):

        input_ids = batch[0].to(device)
        input_mask = batch[1].to(device)
        labels = batch[2].to(device)

        model1.zero_grad()
        out = model1(input_ids = input_ids, attention_mask = input_mask, token_type_ids = None)
        loss = criterion(out, labels)
        total_train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model1.parameters(), 1.0)
        optimizer.step()

        pred = torch.argmax(out, dim = 1)
        total_train_accuracy +=  torch.sum(pred == labels).item()

    avg_train_accuracy = total_train_accuracy / len(train_dataloader2)
    avg_train_loss = total_train_loss / len(train_dataloader2)
    print("  BERT+CNN_20_Words Accuracy: {}".format(avg_train_accuracy))
    print("  Training loss: {}".format(avg_train_loss))

    # Validation
    print("")
    print("Validation...")
    model1.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    y_true = []
    y_pred = []

    for batch in validation_dataloader1:
        input_ids = batch[0].to(device)
        input_mask = batch[1].to(device)
        labels = batch[2].to(device)

        with torch.no_grad():
            out = model1(input_ids = input_ids, attention_mask = input_mask, token_type_ids = None)
        loss = criterion(out, labels)
        total_eval_loss += loss.item()
        pred = torch.argmax(out, dim = 1)
        total_eval_accuracy += torch.sum(pred == labels).item()
        y_true.append(labels.flatten())
        y_pred.append(pred.flatten())

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader2)
    print("  BERT+CNN_20_Words Accuracy: {}".format(avg_val_accuracy))
    avg_val_loss = total_eval_loss / len(validation_dataloader2)
    print("  BERT+CNN_20_Words Validation loss: {}".format(avg_val_loss))
    training_time = format_time(time.time() - t0)
    print("  BERT+CNN_20_Words  This epoch took: {:}".format(training_time))
    print()
    y_true = torch.cat(y_true).tolist()
    y_pred = torch.cat(y_pred).tolist()
    print('  BERT+CNN_20_Words  roc_auc score: ', roc_auc_score(y_true,y_pred))
    print('  BERT+CNN_20_Words  F1 score:',f1_score(y_true, y_pred))


    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Train Accur.': avg_train_accuracy,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
        }
    )

    if avg_val_accuracy > best_accuracy:
        best_accuracy = avg_val_accuracy
        best_model = model1

print("===")
print("Summary")
print("BERT+CNN_20_Words Total time {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
print('BERT+CNN_20_Words best acc:',best_accuracy)


Epoch 1 / 4
Training...
  BERT+CNN_20_Words Accuracy: 14.102473498233216
  Training loss: 0.43195832449640065

Validation...
  BERT+CNN_20_Words Accuracy: 10.971830985915492
  BERT+CNN_20_Words Validation loss: 0.6251021955214756
  BERT+CNN_20_Words  This epoch took: 0:02:21

  BERT+CNN_20_Words  roc_auc score:  0.6006787330316742
  BERT+CNN_20_Words  F1 score: 0.7963069821119446

Epoch 2 / 4
Training...
  BERT+CNN_20_Words Accuracy: 14.328621908127209
  Training loss: 0.416340834049791

Validation...
  BERT+CNN_20_Words Accuracy: 11.549295774647888
  BERT+CNN_20_Words Validation loss: 0.5893861399569982
  BERT+CNN_20_Words  This epoch took: 0:02:19

  BERT+CNN_20_Words  roc_auc score:  0.6470588235294117
  BERT+CNN_20_Words  F1 score: 0.8156028368794326

Epoch 3 / 4
Training...
  BERT+CNN_20_Words Accuracy: 14.378091872791519
  Training loss: 0.41439884290257106

Validation...
  BERT+CNN_20_Words Accuracy: 10.647887323943662
  BERT+CNN_20_Words Validation loss: 0.6447818854325255
  B

BERT+CNN with Thirty_Words Labels

In [24]:
##=====================================================================================
seed_val = 42
random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []
total_t0 = time.time()
best_accuracy = 0
for epoch_i in range(0, epochs):
    #Training
    print("")
    print('Epoch {:} / {:}'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_train_loss = 0
    total_train_accuracy = 0
    model1.train()
    for step, batch in enumerate(train_dataloader3):

        input_ids = batch[0].to(device)
        input_mask = batch[1].to(device)
        labels = batch[2].to(device)

        model1.zero_grad()
        out = model1(input_ids = input_ids, attention_mask = input_mask, token_type_ids = None)
        loss = criterion(out, labels)
        total_train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model1.parameters(), 1.0)
        optimizer.step()

        pred = torch.argmax(out, dim = 1)
        total_train_accuracy +=  torch.sum(pred == labels).item()

    avg_train_accuracy = total_train_accuracy / len(train_dataloader3)
    avg_train_loss = total_train_loss / len(train_dataloader3)
    print("  BERT+CNN_30_Words Accuracy: {}".format(avg_train_accuracy))
    print("  BERT+CNN_30_Words Training loss: {}".format(avg_train_loss))

    # Validation
    print("")
    print("Validation...")
    model1.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    y_true = []
    y_pred = []

    for batch in validation_dataloader1:
        input_ids = batch[0].to(device)
        input_mask = batch[1].to(device)
        labels = batch[2].to(device)

        with torch.no_grad():
            out = model1(input_ids = input_ids, attention_mask = input_mask, token_type_ids = None)
        loss = criterion(out, labels)
        total_eval_loss += loss.item()
        pred = torch.argmax(out, dim = 1)
        total_eval_accuracy += torch.sum(pred == labels).item()
        y_true.append(labels.flatten())
        y_pred.append(pred.flatten())

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader3)
    print("  BERT+CNN_30_Words Accuracy: {}".format(avg_val_accuracy))
    avg_val_loss = total_eval_loss / len(validation_dataloader3)
    print("  BERT+CNN_30_Words Validation loss: {}".format(avg_val_loss))
    training_time = format_time(time.time() - t0)
    print("  BERT+CNN_30_Words This epoch took: {:}".format(training_time))
    print()
    y_true = torch.cat(y_true).tolist()
    y_pred = torch.cat(y_pred).tolist()
    print('  BERT+CNN_30_Words roc_auc score: ', roc_auc_score(y_true,y_pred))
    print('  BERT+CNN_30_Words F1 score:',f1_score(y_true, y_pred))


    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Train Accur.': avg_train_accuracy,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
        }
    )

    if avg_val_accuracy > best_accuracy:
        best_accuracy = avg_val_accuracy
        best_model = model1

print("===")
print("Summary")
print("BERT+CNN_30_Words Total time {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
print('BERT+CNN_30_Words best acc:',best_accuracy)


Epoch 1 / 4
Training...
  BERT+CNN_30_Words Accuracy: 14.809187279151944
  BERT+CNN_30_Words Training loss: 0.3874846046046739

Validation...
  BERT+CNN_30_Words Accuracy: 10.71830985915493
  BERT+CNN_30_Words Validation loss: 0.6411585325086621
  BERT+CNN_30_Words This epoch took: 0:02:20

  BERT+CNN_30_Words roc_auc score:  0.5803167420814479
  BERT+CNN_30_Words F1 score: 0.7881210736721873

Epoch 2 / 4
Training...
  BERT+CNN_30_Words Accuracy: 14.865724381625443
  BERT+CNN_30_Words Training loss: 0.383455978370808

Validation...
  BERT+CNN_30_Words Accuracy: 12.943661971830986
  BERT+CNN_30_Words Validation loss: 0.5021305445214392
  BERT+CNN_30_Words This epoch took: 0:02:19

  BERT+CNN_30_Words roc_auc score:  0.7635221981769297
  BERT+CNN_30_Words F1 score: 0.864417568427753

Epoch 3 / 4
Training...
  BERT+CNN_30_Words Accuracy: 14.46643109540636
  BERT+CNN_30_Words Training loss: 0.40513602941701776

Validation...
  BERT+CNN_30_Words Accuracy: 9.71830985915493
  BERT+CNN_30_Wor

#BERT+Bidirectional LSTM

In [25]:
class BertLstmClassifier(nn.Module):
    def __init__(self, model_tune):
        super().__init__()
        self.bert = model_tune.bert
        self.lstm = nn.LSTM(input_size = 768,
                            hidden_size = 768,
                            num_layers = 1,
                            batch_first = True,
                            bidirectional = True)
        self.classifier = nn.Linear(768 * 2, 2)
        self.softmax = nn.Softmax(dim = 1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        bert_output = self.bert(input_ids = input_ids, attention_mask = attention_mask, token_type_ids = token_type_ids)
        out, _ = self.lstm(bert_output[0])
        logits = self.classifier(out[:, 1, :])
        return self.softmax(logits)

In [26]:

# Initializing model
model2 = BertCNNClassifier(tuned_model=tuned_model)
model2.to(device)
# set parameters
epochs = 4
learning_rate = 5e-5
optimizer = AdamW(model2.parameters(), lr = learning_rate)
criterion = nn.CrossEntropyLoss()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


BERT+BiLSTM with Ten_words Labels

In [27]:
seed_val = 42

random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []
total_t0 = time.time()
best_accuracy = 0
for epoch_i in range(0, epochs):
    #Training
    print("")
    print('Epoch {:} / {:}'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_train_loss = 0
    total_train_accuracy = 0
    model2.train()
    for step, batch in enumerate(train_dataloader1):

        input_ids = batch[0].to(device)
        input_mask = batch[1].to(device)
        labels = batch[2].to(device)

        model2.zero_grad()
        out = model2(input_ids = input_ids, attention_mask = input_mask, token_type_ids = None)
        loss = criterion(out, labels)
        total_train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model2.parameters(), 1.0)
        optimizer.step()

        pred = torch.argmax(out, dim = 1)
        total_train_accuracy +=  torch.sum(pred == labels).item()

    avg_train_accuracy = total_train_accuracy / len(train_dataloader1)
    avg_train_loss = total_train_loss / len(train_dataloader1)
    print("  BERT+BiLSTM_10_words Accuracy: {}".format(avg_train_accuracy))
    print("  BERT+BiLSTM_10_words Training loss: {}".format(avg_train_loss))

    # Validation
    print("")
    print("Validation...")
    model2.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    y_true = []
    y_pred = []

    for batch in validation_dataloader1:
        input_ids = batch[0].to(device)
        input_mask = batch[1].to(device)
        labels = batch[2].to(device)

        with torch.no_grad():
            out = model2(input_ids = input_ids, attention_mask = input_mask, token_type_ids = None)
        loss = criterion(out, labels)
        total_eval_loss += loss.item()
        pred = torch.argmax(out, dim = 1)
        total_eval_accuracy += torch.sum(pred == labels).item()
        y_true.append(labels.flatten())
        y_pred.append(pred.flatten())

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader1)
    print("  BERT+BiLSTM_10_words Accuracy: {}".format(avg_val_accuracy))
    avg_val_loss = total_eval_loss / len(validation_dataloader1)
    print("  BERT+BiLSTM_10_words Validation loss: {}".format(avg_val_loss))
    training_time = format_time(time.time() - t0)
    print("  BERT+BiLSTM_10_words This epoch took: {:}".format(training_time))
    print()
    y_true = torch.cat(y_true).tolist()
    y_pred = torch.cat(y_pred).tolist()
    print('  BERT+BiLSTM_10_words roc_auc score: ', roc_auc_score(y_true,y_pred))
    print('  BERT+BiLSTM_10_words F1 score:',f1_score(y_true, y_pred))


    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Train Accur.': avg_train_accuracy,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
        }
    )

    if avg_val_accuracy > best_accuracy:
        best_accuracy = avg_val_accuracy
        best_model = model2

print("===")
print("Summary")
print("BERT+BiLSTM_10_words Total time {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
print('BERT+BiLSTM_10_words best acc:',best_accuracy)


Epoch 1 / 4
Training...
  BERT+BiLSTM_10_words Accuracy: 13.257950530035336
  BERT+BiLSTM_10_words Training loss: 0.48920736885744776

Validation...
  BERT+BiLSTM_10_words Accuracy: 12.690140845070422
  BERT+BiLSTM_10_words Validation loss: 0.5189936555607219
  BERT+BiLSTM_10_words This epoch took: 0:02:17

  BERT+BiLSTM_10_words roc_auc score:  0.7423470391501081
  BERT+BiLSTM_10_words F1 score: 0.8549905838041432

Epoch 2 / 4
Training...
  BERT+BiLSTM_10_words Accuracy: 13.137809187279151
  BERT+BiLSTM_10_words Training loss: 0.49125547775531403

Validation...
  BERT+BiLSTM_10_words Accuracy: 13.380281690140846
  BERT+BiLSTM_10_words Validation loss: 0.4919324748113122
  BERT+BiLSTM_10_words This epoch took: 0:02:17

  BERT+BiLSTM_10_words roc_auc score:  0.8538855006885698
  BERT+BiLSTM_10_words F1 score: 0.8564668769716088

Epoch 3 / 4
Training...
  BERT+BiLSTM_10_words Accuracy: 10.197879858657243
  BERT+BiLSTM_10_words Training loss: 0.5698116766058514

Validation...
  BERT+BiLS

BERT+BiLSTM with Twenty_Words Labels

In [28]:
seed_val = 42

random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []
total_t0 = time.time()
best_accuracy = 0
for epoch_i in range(0, epochs):
    #Training
    print("")
    print('Epoch {:} / {:}'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_train_loss = 0
    total_train_accuracy = 0
    model2.train()
    for step, batch in enumerate(train_dataloader2):

        input_ids = batch[0].to(device)
        input_mask = batch[1].to(device)
        labels = batch[2].to(device)

        model2.zero_grad()
        out = model2(input_ids = input_ids, attention_mask = input_mask, token_type_ids = None)
        loss = criterion(out, labels)
        total_train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model2.parameters(), 1.0)
        optimizer.step()

        pred = torch.argmax(out, dim = 1)
        total_train_accuracy +=  torch.sum(pred == labels).item()

    avg_train_accuracy = total_train_accuracy / len(train_dataloader2)
    avg_train_loss = total_train_loss / len(train_dataloader2)
    print("  BERT+BiLSTM_20_words Accuracy: {}".format(avg_train_accuracy))
    print("  BERT+BiLSTM_20_words Training loss: {}".format(avg_train_loss))

    # Validation
    print("")
    print("Validation...")
    model2.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    y_true = []
    y_pred = []

    for batch in validation_dataloader1:
        input_ids = batch[0].to(device)
        input_mask = batch[1].to(device)
        labels = batch[2].to(device)

        with torch.no_grad():
            out = model2(input_ids = input_ids, attention_mask = input_mask, token_type_ids = None)
        loss = criterion(out, labels)
        total_eval_loss += loss.item()
        pred = torch.argmax(out, dim = 1)
        total_eval_accuracy += torch.sum(pred == labels).item()
        y_true.append(labels.flatten())
        y_pred.append(pred.flatten())

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader2)
    print("  BERT+BiLSTM_20_words Accuracy: {}".format(avg_val_accuracy))
    avg_val_loss = total_eval_loss / len(validation_dataloader2)
    print("  BERT+BiLSTM_20_words Validation loss: {}".format(avg_val_loss))
    training_time = format_time(time.time() - t0)
    print("  BERT+BiLSTM_20_words This epoch took: {:}".format(training_time))
    print()
    y_true = torch.cat(y_true).tolist()
    y_pred = torch.cat(y_pred).tolist()
    print('  BERT+BiLSTM_20_words roc_auc score: ', roc_auc_score(y_true,y_pred))
    print('  BERT+BiLSTM_20_words F1 score:',f1_score(y_true, y_pred))


    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Train Accur.': avg_train_accuracy,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
        }
    )

    if avg_val_accuracy > best_accuracy:
        best_accuracy = avg_val_accuracy
        best_model = model2

print("===")
print("Summary")
print("BERT+BiLSTM_20_words Total time {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
print('BERT+BiLSTM_20_words best acc:',best_accuracy)


Epoch 1 / 4
Training...
  BERT+BiLSTM_20_words Accuracy: 13.522968197879859
  BERT+BiLSTM_20_words Training loss: 0.47198258019167627

Validation...
  BERT+BiLSTM_20_words Accuracy: 9.71830985915493
  BERT+BiLSTM_20_words Validation loss: 0.5750925146358113
  BERT+BiLSTM_20_words This epoch took: 0:02:15

  BERT+BiLSTM_20_words roc_auc score:  0.5
  BERT+BiLSTM_20_words F1 score: 0.7574094401756313

Epoch 2 / 4
Training...
  BERT+BiLSTM_20_words Accuracy: 13.522968197879859
  BERT+BiLSTM_20_words Training loss: 0.43634380068458856

Validation...
  BERT+BiLSTM_20_words Accuracy: 9.71830985915493
  BERT+BiLSTM_20_words Validation loss: 0.6179487512145244
  BERT+BiLSTM_20_words This epoch took: 0:02:16

  BERT+BiLSTM_20_words roc_auc score:  0.5
  BERT+BiLSTM_20_words F1 score: 0.7574094401756313

Epoch 3 / 4
Training...
  BERT+BiLSTM_20_words Accuracy: 13.522968197879859
  BERT+BiLSTM_20_words Training loss: 0.44437932188856305

Validation...
  BERT+BiLSTM_20_words Accuracy: 9.718309859

BERT+BiLSTM with Thirty_Words Labels

In [29]:
seed_val = 42

random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []
total_t0 = time.time()
best_accuracy = 0
for epoch_i in range(0, epochs):
    #Training
    print("")
    print('Epoch {:} / {:}'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_train_loss = 0
    total_train_accuracy = 0
    model2.train()
    for step, batch in enumerate(train_dataloader3):

        input_ids = batch[0].to(device)
        input_mask = batch[1].to(device)
        labels = batch[2].to(device)

        model2.zero_grad()
        out = model2(input_ids = input_ids, attention_mask = input_mask, token_type_ids = None)
        loss = criterion(out, labels)
        total_train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model2.parameters(), 1.0)
        optimizer.step()

        pred = torch.argmax(out, dim = 1)
        total_train_accuracy +=  torch.sum(pred == labels).item()

    avg_train_accuracy = total_train_accuracy / len(train_dataloader3)
    avg_train_loss = total_train_loss / len(train_dataloader3)
    print("  BERT+BiLSTM_30_words Accuracy: {}".format(avg_train_accuracy))
    print("  BERT+BiLSTM_30_words Training loss: {}".format(avg_train_loss))

    # Validation
    print("")
    print("Validation...")
    model2.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    y_true = []
    y_pred = []

    for batch in validation_dataloader1:
        input_ids = batch[0].to(device)
        input_mask = batch[1].to(device)
        labels = batch[2].to(device)

        with torch.no_grad():
            out = model2(input_ids = input_ids, attention_mask = input_mask, token_type_ids = None)
        loss = criterion(out, labels)
        total_eval_loss += loss.item()
        pred = torch.argmax(out, dim = 1)
        total_eval_accuracy += torch.sum(pred == labels).item()
        y_true.append(labels.flatten())
        y_pred.append(pred.flatten())

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader3)
    print("  BERT+BiLSTM_30_words Accuracy: {}".format(avg_val_accuracy))
    avg_val_loss = total_eval_loss / len(validation_dataloader3)
    print("  BERT+BiLSTM_30_words Validation loss: {}".format(avg_val_loss))
    training_time = format_time(time.time() - t0)
    print("  BERT+BiLSTM_30_words This epoch took: {:}".format(training_time))
    print()
    y_true = torch.cat(y_true).tolist()
    y_pred = torch.cat(y_pred).tolist()
    print('  BERT+BiLSTM_30_words roc_auc score: ', roc_auc_score(y_true,y_pred))
    print('  BERT+BiLSTM_30_words F1 score:',f1_score(y_true, y_pred))


    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Train Accur.': avg_train_accuracy,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
        }
    )

    if avg_val_accuracy > best_accuracy:
        best_accuracy = avg_val_accuracy
        best_model = model2

print("===")
print("Summary")
print("BERT+BiLSTM_30_words Total time {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
print('BERT+BiLSTM_30_words best acc:',best_accuracy)


Epoch 1 / 4
Training...
  BERT+BiLSTM_30_words Accuracy: 14.558303886925795
  BERT+BiLSTM_30_words Training loss: 0.3970156261440722

Validation...
  BERT+BiLSTM_30_words Accuracy: 9.71830985915493
  BERT+BiLSTM_30_words Validation loss: 0.6271419256505831
  BERT+BiLSTM_30_words This epoch took: 0:02:16

  BERT+BiLSTM_30_words roc_auc score:  0.5
  BERT+BiLSTM_30_words F1 score: 0.7574094401756313

Epoch 2 / 4
Training...
  BERT+BiLSTM_30_words Accuracy: 14.558303886925795
  BERT+BiLSTM_30_words Training loss: 0.3960857876829882

Validation...
  BERT+BiLSTM_30_words Accuracy: 9.71830985915493
  BERT+BiLSTM_30_words Validation loss: 0.6282645031599932
  BERT+BiLSTM_30_words This epoch took: 0:02:16

  BERT+BiLSTM_30_words roc_auc score:  0.5
  BERT+BiLSTM_30_words F1 score: 0.7574094401756313

Epoch 3 / 4
Training...
  BERT+BiLSTM_30_words Accuracy: 14.558303886925795
  BERT+BiLSTM_30_words Training loss: 0.394175230403671

Validation...
  BERT+BiLSTM_30_words Accuracy: 9.7183098591549

In [30]:
class BertCNNBiLSTMClassifier(nn.Module):
    def __init__(self, tuned_model, embed_num = 512, embed_dim = 768, dropout=0.1, kernel_num=3, kernel_sizes=[1,2], num_labels=2):
        super().__init__()
        self.num_labels = num_labels
        self.embed_num = embed_num
        self.embed_dim = embed_dim
        self.dropout = dropout
        self.kernel_num = kernel_num
        self.kernel_sizes = kernel_sizes
        self.softmax = nn.functional.softmax

        self.bert = tuned_model.bert
        self.convs = nn.ModuleList([nn.Conv2d(1, self.kernel_num, (k, self.embed_dim)) for k in self.kernel_sizes])
        self.dropout = nn.Dropout(self.dropout)
        self.lstms = nn.LSTM(input_size = 768, hidden_size = 768, num_layers = 1, batch_first = True, bidirectional = True)
        self.classifier = nn.Linear(len(self.kernel_sizes)*self.kernel_num, self.num_labels)

    def forward(self, input_ids, attention_mask, token_type_ids = None):
        output = self.bert(input_ids = input_ids, attention_mask = attention_mask, token_type_ids = token_type_ids) #16,512,768
        output = output[0].unsqueeze(1) #16,1,512,768
        output = [nn.functional.relu(conv(output)).squeeze(3) for conv in self.convs] #16,3,508,1 => #16,3,508
        output = [nn.functional.max_pool1d(i, i.size(2)).squeeze(2) for i in output] #=> 16,3
        output = torch.cat(output, 1)
        output = self.dropout(output)
        logits = self.classifier(output)
        return self.softmax(logits, 1)



# Initializing model
model3 = BertCNNBiLSTMClassifier(tuned_model=tuned_model)
model3.to(device)
# set parameters
epochs = 4
learning_rate = 5e-5
optimizer = AdamW(model3.parameters(), lr = learning_rate)
criterion = nn.CrossEntropyLoss()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


BERT+CNNBiLSTM with Ten_Words Labels

In [31]:
seed_val = 42

random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []
total_t0 = time.time()
best_accuracy = 0
for epoch_i in range(0, epochs):
    #Training
    print("")
    print('Epoch {:} / {:}'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_train_loss = 0
    total_train_accuracy = 0
    model3.train()
    for step, batch in enumerate(train_dataloader1):

        input_ids = batch[0].to(device)
        input_mask = batch[1].to(device)
        labels = batch[2].to(device)

        model3.zero_grad()
        out = model3(input_ids = input_ids, attention_mask = input_mask, token_type_ids = None)
        loss = criterion(out, labels)
        total_train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model3.parameters(), 1.0)
        optimizer.step()

        pred = torch.argmax(out, dim = 1)
        total_train_accuracy +=  torch.sum(pred == labels).item()

    avg_train_accuracy = total_train_accuracy / len(train_dataloader1)
    avg_train_loss = total_train_loss / len(train_dataloader1)
    print("  BERT+CNNBiLSTM_10_words Accuracy: {}".format(avg_train_accuracy))
    print("  BERT+CNNBiLSTM_10_words Training loss: {}".format(avg_train_loss))

    # Validation
    print("")
    print("Validation...")
    model3.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    y_true = []
    y_pred = []

    for batch in validation_dataloader1:
        input_ids = batch[0].to(device)
        input_mask = batch[1].to(device)
        labels = batch[2].to(device)

        with torch.no_grad():
            out = model3(input_ids = input_ids, attention_mask = input_mask, token_type_ids = None)
        loss = criterion(out, labels)
        total_eval_loss += loss.item()
        pred = torch.argmax(out, dim = 1)
        total_eval_accuracy += torch.sum(pred == labels).item()
        y_true.append(labels.flatten())
        y_pred.append(pred.flatten())

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader1)
    print("  BERT+CNNBiLSTM_10_words Accuracy: {}".format(avg_val_accuracy))
    avg_val_loss = total_eval_loss / len(validation_dataloader1)
    print("  BERT+CNNBiLSTM_10_words Validation loss: {}".format(avg_val_loss))
    training_time = format_time(time.time() - t0)
    print("  BERT+CNNBiLSTM_10_words This epoch took: {:}".format(training_time))
    print()
    y_true = torch.cat(y_true).tolist()
    y_pred = torch.cat(y_pred).tolist()
    print('  BERT+CNNBiLSTM_10_words roc_auc score: ', roc_auc_score(y_true,y_pred))
    print('  BERT+CNNBiLSTM_10_words F1 score:',f1_score(y_true, y_pred))


    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Train Accur.': avg_train_accuracy,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
        }
    )

    if avg_val_accuracy > best_accuracy:
        best_accuracy = avg_val_accuracy
        best_model = model3

print("===")
print("Summary")
print("BERT+CNNBiLSTM_10_words Total time {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
print('BERT+CNNBiLSTM_10_words best acc:',best_accuracy)


Epoch 1 / 4
Training...
  BERT+CNNBiLSTM_10_words Accuracy: 12.731448763250883
  BERT+CNNBiLSTM_10_words Training loss: 0.5941776141471661

Validation...
  BERT+CNNBiLSTM_10_words Accuracy: 13.253521126760564
  BERT+CNNBiLSTM_10_words Validation loss: 0.551092160419679
  BERT+CNNBiLSTM_10_words This epoch took: 0:02:16

  BERT+CNNBiLSTM_10_words roc_auc score:  0.8140238704177323
  BERT+CNNBiLSTM_10_words F1 score: 0.8657765284609978

Epoch 2 / 4
Training...
  BERT+CNNBiLSTM_10_words Accuracy: 13.374558303886927
  BERT+CNNBiLSTM_10_words Training loss: 0.5502783783543658

Validation...
  BERT+CNNBiLSTM_10_words Accuracy: 13.140845070422536
  BERT+CNNBiLSTM_10_words Validation loss: 0.5506386236405708
  BERT+CNNBiLSTM_10_words This epoch took: 0:02:16

  BERT+CNNBiLSTM_10_words roc_auc score:  0.7891173191684702
  BERT+CNNBiLSTM_10_words F1 score: 0.8681245858184228

Epoch 3 / 4
Training...
  BERT+CNNBiLSTM_10_words Accuracy: 12.307420494699647
  BERT+CNNBiLSTM_10_words Training loss: 

BERT+CNNBiLSTM with Twenty_Words Labels

In [32]:
seed_val = 42
random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []
total_t0 = time.time()
best_accuracy = 0
for epoch_i in range(0, epochs):
    #Training
    print("")
    print('Epoch {:} / {:}'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_train_loss = 0
    total_train_accuracy = 0
    model3.train()
    for step, batch in enumerate(train_dataloader2):

        input_ids = batch[0].to(device)
        input_mask = batch[1].to(device)
        labels = batch[2].to(device)

        model3.zero_grad()
        out = model3(input_ids = input_ids, attention_mask = input_mask, token_type_ids = None)
        loss = criterion(out, labels)
        total_train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model3.parameters(), 1.0)
        optimizer.step()

        pred = torch.argmax(out, dim = 1)
        total_train_accuracy +=  torch.sum(pred == labels).item()

    avg_train_accuracy = total_train_accuracy / len(train_dataloader2)
    avg_train_loss = total_train_loss / len(train_dataloader2)
    print("  BERT+CNNBiLSTM_20_words Accuracy: {}".format(avg_train_accuracy))
    print("  BERT+CNNBiLSTM_20_words Training loss: {}".format(avg_train_loss))

    # Validation
    print("")
    print("Validation...")
    model3.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    y_true = []
    y_pred = []

    for batch in validation_dataloader1:
        input_ids = batch[0].to(device)
        input_mask = batch[1].to(device)
        labels = batch[2].to(device)

        with torch.no_grad():
            out = model3(input_ids = input_ids, attention_mask = input_mask, token_type_ids = None)
        loss = criterion(out, labels)
        total_eval_loss += loss.item()
        pred = torch.argmax(out, dim = 1)
        total_eval_accuracy += torch.sum(pred == labels).item()
        y_true.append(labels.flatten())
        y_pred.append(pred.flatten())

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader2)
    print("  BERT+CNNBiLSTM_20_words Accuracy: {}".format(avg_val_accuracy))
    avg_val_loss = total_eval_loss / len(validation_dataloader2)
    print("  BERT+CNNBiLSTM_20_words Validation loss: {}".format(avg_val_loss))
    training_time = format_time(time.time() - t0)
    print("  BERT+CNNBiLSTM_20_words This epoch took: {:}".format(training_time))
    print()
    y_true = torch.cat(y_true).tolist()
    y_pred = torch.cat(y_pred).tolist()
    print('  BERT+CNNBiLSTM_20_words roc_auc score: ', roc_auc_score(y_true,y_pred))
    print('  BERT+CNNBiLSTM_20_words F1 score:',f1_score(y_true, y_pred))


    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Train Accur.': avg_train_accuracy,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
        }
    )

    if avg_val_accuracy > best_accuracy:
        best_accuracy = avg_val_accuracy
        best_model = model3

print("===")
print("Summary")
print("BERT+CNNBiLSTM_20_words Total time {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
print('BERT+CNNBiLSTM_20_words best acc:',best_accuracy)


Epoch 1 / 4
Training...
  BERT+CNNBiLSTM_20_words Accuracy: 10.06713780918728
  BERT+CNNBiLSTM_20_words Training loss: 0.528938250701756

Validation...
  BERT+CNNBiLSTM_20_words Accuracy: 13.422535211267606
  BERT+CNNBiLSTM_20_words Validation loss: 0.5396412287799406
  BERT+CNNBiLSTM_20_words This epoch took: 0:02:15

  BERT+CNNBiLSTM_20_words roc_auc score:  0.8206866024001573
  BERT+CNNBiLSTM_20_words F1 score: 0.8761245674740484

Epoch 2 / 4
Training...
  BERT+CNNBiLSTM_20_words Accuracy: 12.773851590106007
  BERT+CNNBiLSTM_20_words Training loss: 0.4582382494274382

Validation...
  BERT+CNNBiLSTM_20_words Accuracy: 12.591549295774648
  BERT+CNNBiLSTM_20_words Validation loss: 0.5824779494547508
  BERT+CNNBiLSTM_20_words This epoch took: 0:02:15

  BERT+CNNBiLSTM_20_words roc_auc score:  0.7376811594202899
  BERT+CNNBiLSTM_20_words F1 score: 0.8497474747474748

Epoch 3 / 4
Training...
  BERT+CNNBiLSTM_20_words Accuracy: 13.480565371024735
  BERT+CNNBiLSTM_20_words Training loss: 0

BERT+CNNBiLSTM with Thirty_Words Labels

In [33]:
seed_val = 42

random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []
total_t0 = time.time()
best_accuracy = 0
for epoch_i in range(0, epochs):
    #Training
    print("")
    print('Epoch {:} / {:}'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_train_loss = 0
    total_train_accuracy = 0
    model3.train()
    for step, batch in enumerate(train_dataloader3):

        input_ids = batch[0].to(device)
        input_mask = batch[1].to(device)
        labels = batch[2].to(device)

        model3.zero_grad()
        out = model3(input_ids = input_ids, attention_mask = input_mask, token_type_ids = None)
        loss = criterion(out, labels)
        total_train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model3.parameters(), 1.0)
        optimizer.step()

        pred = torch.argmax(out, dim = 1)
        total_train_accuracy +=  torch.sum(pred == labels).item()

    avg_train_accuracy = total_train_accuracy / len(train_dataloader3)
    avg_train_loss = total_train_loss / len(train_dataloader3)
    print("  BERT+CNNBiLSTM_30_words Accuracy: {}".format(avg_train_accuracy))
    print("  BERT+CNNBiLSTM_30_words Training loss: {}".format(avg_train_loss))

    # Validation
    print("")
    print("Validation...")
    model3.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    y_true = []
    y_pred = []

    for batch in validation_dataloader3:
        input_ids = batch[0].to(device)
        input_mask = batch[1].to(device)
        labels = batch[2].to(device)

        with torch.no_grad():
            out = model3(input_ids = input_ids, attention_mask = input_mask, token_type_ids = None)
        loss = criterion(out, labels)
        total_eval_loss += loss.item()
        pred = torch.argmax(out, dim = 1)
        total_eval_accuracy += torch.sum(pred == labels).item()
        y_true.append(labels.flatten())
        y_pred.append(pred.flatten())

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader3)
    print("  BERT+CNNBiLSTM_30_words Accuracy: {}".format(avg_val_accuracy))
    avg_val_loss = total_eval_loss / len(validation_dataloader3)
    print("  BERT+CNNBiLSTM_30_words Validation loss: {}".format(avg_val_loss))
    training_time = format_time(time.time() - t0)
    print("  BERT+CNNBiLSTM_30_words This epoch took: {:}".format(training_time))
    print()
    y_true = torch.cat(y_true).tolist()
    y_pred = torch.cat(y_pred).tolist()
    print('  BERT+CNNBiLSTM_30_words roc_auc score: ', roc_auc_score(y_true,y_pred))
    print('  BERT+CNNBiLSTM_30_words F1 score:',f1_score(y_true, y_pred))


    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Train Accur.': avg_train_accuracy,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
        }
    )

    if avg_val_accuracy > best_accuracy:
        best_accuracy = avg_val_accuracy
        best_model = model3

print("===")
print("Summary")
print("BERT+CNNBiLSTM_30_words Total time {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
print('BERT+CNNBiLSTM_30_words best acc:',best_accuracy)


Epoch 1 / 4
Training...
  BERT+CNNBiLSTM_30_words Accuracy: 14.784452296819788
  BERT+CNNBiLSTM_30_words Training loss: 0.38979262545336263

Validation...
  BERT+CNNBiLSTM_30_words Accuracy: 15.0
  BERT+CNNBiLSTM_30_words Validation loss: 0.3768717424130776
  BERT+CNNBiLSTM_30_words This epoch took: 0:02:16

  BERT+CNNBiLSTM_30_words roc_auc score:  0.7200877926421405
  BERT+CNNBiLSTM_30_words F1 score: 0.9682915286322763

Epoch 2 / 4
Training...
  BERT+CNNBiLSTM_30_words Accuracy: 14.840989399293287
  BERT+CNNBiLSTM_30_words Training loss: 0.3880264783284689

Validation...
  BERT+CNNBiLSTM_30_words Accuracy: 15.028169014084508
  BERT+CNNBiLSTM_30_words Validation loss: 0.3784017944839639
  BERT+CNNBiLSTM_30_words This epoch took: 0:02:16

  BERT+CNNBiLSTM_30_words roc_auc score:  0.6665551839464883
  BERT+CNNBiLSTM_30_words F1 score: 0.969583528310716

Epoch 3 / 4
Training...
  BERT+CNNBiLSTM_30_words Accuracy: 14.791519434628976
  BERT+CNNBiLSTM_30_words Training loss: 0.39238613159